In [1]:
from scipy.stats import hypergeom

In [2]:
def calculate_probability(V, C):
    """
    Calculate the probability of drawing more than 2/3 malicious validators
    in a committee of size C from a total validator set of size V.

    :param V: Total number of validators
    :param C: Committee size
    :return: Probability
    """
    malicious = V // 3  # Number of malicious validators (1/3 of total)
    threshold = int(2 * C // 3)  # Threshold for malicious majority

    # Calculate the probability of having more than threshold malicious validators
    prob = 1 - hypergeom.cdf(threshold, V, malicious, C)

    return prob



In [3]:
def calculate_first_p_malicious(V, C, P):
    """
    Calculate the probability that the first P validators in a randomly ordered
    committee of size C (drawn from a total set of V validators) are all malicious.

    :param V: Total number of validators
    :param C: Committee size
    :param P: Number of first validators to check
    :return: Probability
    """
    malicious = V // 3  # Number of malicious validators (1/3 of total)

    # Probability of exactly k malicious validators in the committee
    prob_k_malicious = lambda k: hypergeom.pmf(k, V, malicious, C)

    total_prob = 0
    for k in range(P, C + 1):  # k cannot be less than P
        prob_committee = prob_k_malicious(k)
        prob_first_p_malicious = 1
        for i in range(P):
            prob_first_p_malicious *= (k - i) / (C - i)
        total_prob += prob_committee * prob_first_p_malicious

    return total_prob

In [4]:
def find_smallest_C(V, threshold=1e-6):
    """
    Find the smallest C such that the probability of more than 2/3 of the
    committee being malicious is less than the threshold.

    :param V: Total number of validators
    :param threshold: Probability threshold
    :return: Smallest C meeting the criteria
    """
    C = 1
    while calculate_probability(V, C) >= threshold:
        C += 1
    return C


def find_smallest_P(V, C, threshold=1e-6):
    """
    Find the smallest P such that the probability of the first P validators
    in a row being malicious is less than the threshold.

    :param V: Total number of validators
    :param C: Committee size
    :param threshold: Probability threshold
    :return: Smallest P meeting the criteria
    """
    P = 1
    while calculate_first_p_malicious(V, C, P) >= threshold:
        P += 1
    return P

In [5]:
# Example usage
V = 10000  # Total number of validators
threshold = 1e-6

smallest_C = find_smallest_C(V, threshold)
smallest_P = find_smallest_P(V, smallest_C, threshold)

print(f"For V = {V}:")
print(f"Smallest C: {smallest_C}")
print(
    f"Probability of malicious majority for C = {smallest_C}: {calculate_probability(V, smallest_C):.2e}"
)
print(f"Smallest P for C = {smallest_C}: {smallest_P}")
print(
    f"Probability of first {smallest_P} being malicious: {calculate_first_p_malicious(V, smallest_C, smallest_P):.2e}"
)

For V = 10000:
Smallest C: 48
Probability of malicious majority for C = 48: 5.35e-07
Smallest P for C = 48: 13
Probability of first 13 being malicious: 6.17e-07
